In [1]:
pip install torch===1.7.0 torchvision===0.8.1 torchaudio===0.7.0 -f https://download.pytorch.org/whl/torch_stable.html


Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 776.7MB 24kB/s 
     |████████████████████████████████| 12.8MB 19.7MB/s 
     |████████████████████████████████| 7.6MB 51.1MB/s 
  Found existing installation: torch 1.6.0+cu101
    Uninstalling torch-1.6.0+cu101:
      Successfully uninstalled torch-1.6.0+cu101
  Found existing installation: torchvision 0.7.0+cu101
    Uninstalling torchvision-0.7.0+cu101:
      Successfully uninstalled torchvision-0.7.0+cu101


In [3]:
import gym
import math
import random
import numpy as np
import matplotlib.pyplot as plt
from collections import namedtuple
from itertools import count
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T
%matplotlib inline

In [5]:
import matplotlib
is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython: from IPython import display

In [6]:
class DQN(nn.Module):
  def __init__(self, img_height, img_width):
    super.__init__()

    self.fc1 = nn.Linear(in_features = img_height*img_width*3, out_features = 24)
    self.fc2 = nn.Linear(in_features = 24, out_features = 32)
    self.out = nn.Linear(in_features = 32, out_features = 2)
  
  def forward(self, t):
    t = t.flatten(start_dim = 1)
    t = F.relu(self.fc1(t))
    t = F.relu(self.fc2(t))
    t = self.out(t)
    
    return t

In [7]:
Experience = namedtuple(
    'Experience',
    ('state', 'action', 'next_state', 'reward')
)

In [8]:
class ReplayMemory():
  def __init__(self, capacity):
    self.capacity = capacity
    self.memory = []
    self.push_count = 0

  def push(self, experience):
    if len(self.memory) < capacity :
      self.memory.append(experience)
    else:
      self.memory[ self.push_count % self.capacity] = experience
    self.push_count+=1

  def sample(self, batch_size):
    return random.sample(self.memory, batch_size)

  def can_provide_sample(self, batch_size):
    return len(self.memory) >= batch_size

In [9]:
class EpsilonGreedyStratergy():
  def __init__(self, start, end, decay):
    self.start = start
    self.end = end
    self.decay = decay

  def get_exploration_rate(self, current_step):
    return self.end + (self.end + self.start) * math.exp(-1 * self.decay * current_step)

In [10]:
class Agent():
  def __init__(self, stratergy, num_action):
    self.current_step = 0
    self.stratergy = stratergy
    self.num_action = num_action

  def select_action(self, state, policy_net):
    rate = self.stratergy.get_exploration_rate(current_step)
    self.current_step+=1

    if rate > random.random():
      return random.randrange(self.num_action)
    else:
      with torch.no_grad():
        return policy_net(state).argmax(dim = 1).item()